In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest
from pyspark.sql import SparkSession

# Tạo Spark session
spark = SparkSession.builder.appName("AnomalyDetection").getOrCreate()

# Đọc file CSV bằng Spark
file_path = "../../../data/zone_temparature_and_humidity.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)

# Chuyển đổi DataFrame của Spark thành Pandas DataFrame
pandas_df = df.toPandas()
column_timestamp= "timeStamp"
# Hiển thị dữ liệu ban đầu
print("Dữ liệu ban đầu:")
print(pandas_df.head())

# Chọn đặc trưng cụ thể để phân tích
feature = 'payload_Zone1_humidity_mt0708_dew_value'
data_feature = pandas_df[feature].dropna().values.reshape(-1, 1)

# Áp dụng phương pháp Isolation Forest
iso_forest = IsolationForest(contamination=0.01, random_state=42)
anomalies = iso_forest.fit_predict(data_feature)
anomaly_indices = np.where(anomalies == -1)[0]

# Vẽ biểu đồ dữ liệu với các điểm bất thường được đánh dấu
plt.figure(figsize=(14, 7))
plt.plot(pandas_df[column_timestamp], pandas_df[feature], label=feature)
plt.scatter(pandas_df.iloc[anomaly_indices][column_timestamp], pandas_df.iloc[anomaly_indices][feature], color='red', label='Anomaly')
plt.xlabel(column_timestamp)
plt.ylabel(feature)
plt.title(f'{feature} Data Over Time with Anomalies Highlighted')
plt.legend()
plt.show()


ConnectionRefusedError: [Errno 61] Connection refused